# OCOM510M Data Science: Assessment 2

Student ID: tpqw3055


## 1. Aims, objectives and plan
The aim of the project is to help the insurance company to predict which claims are probably fradulent by training an Machine Learning models using previus data collected with a low and balanced error rate.
This model can significantly helps the company to reduce the finanantial losses caused by fraud while minimizing customer dissatisfaction from incorrect referrals.

The Objectives is splited in many stages.
1: Underestanding and exploring the data sets 
2, Pre-process the datasets 
3, Apply first ML techniques 
4, Apply the second ML technique
5, Evaluate the results and its performance 
6, Sees the business impact
7, conclusion 





## 2. Understanding the case study

###  Case study analysis
State the key points that you found in the case and how you intend to deal with them appropriately to address the client's needs. (You can include more than four points.) 200-300 words.

1.
1. 
1. 
1. 



## 3. Pre-processing applied

**Create a new subheading for each stage that you do from the following items. Enter your code in the cells below the subheading.**

<!-- - Merging, pivoting and melting, if necessary -->

<!-- - Preparing the labels appropriately, if necessary -->

- Dealing with missing values (imputation, filtering) without leaking, if necessary

- Dealing with duplicate values, if necessary
- Scaling, without leaking, if necessary
- Dealing with correlation and collinearity, if necessary
- Variance analysis, if necessary
<!-- - Appropriate feature selection such as RFE, if necessary -->
- Appropriate feature extraction, if necessary
- Identifying and dealing with class imbalance, if necessary
- Identifying and dealing with outliers, if necessary
- Categorical and numerical encoding if necessary
- Other pre-processing 

### Importing needed libraries and files

Import Libraries

In [136]:
#import the libraries
# For data manipulation
import pandas as pd
import numpy as np

# For data visualization
import matplotlib.pyplot as plt

Import Datasets by pandas 

In [137]:
# Import the datasets

Train_Claim = pd.read_csv(r'D:\Users\ahmed.aldarmaki\Downloads\AssesmentFolder\archive\TrainData\TrainData\Train_Claim.csv')
Train_Demographics = pd.read_csv(r'archive\TrainData\TrainData\Train_Demographics.csv')
Train_Policy = pd.read_csv(r'archive\TrainData\TrainData\Train_Policy.csv')
Train_Vehicle = pd.read_csv(r'archive\TrainData\TrainData\Train_Vehicle.csv')
Traindata_with_Target = pd.read_csv(r'archive\TrainData\TrainData\Traindata_with_Target.csv')
Traindata_withoutTarget = pd.read_csv(r'archive\TrainData\TrainData\Traindata_withoutTarget.csv')

Check if there are mismatching between number of customurs in each dataset

In [138]:
print("Demographics:",Train_Demographics.shape[0])  
print("Policy:",Train_Policy.shape[0])
print("Claim:",Train_Claim.shape[0])  
print("Train_Vehicle:",Train_Vehicle.shape[0]) 
print("with_Target:",Traindata_with_Target.shape[0])  
print("withoutTarget:",Traindata_withoutTarget.shape[0])  


Demographics: 28836
Policy: 28836
Claim: 28836
Train_Vehicle: 115344
with_Target: 28836
withoutTarget: 28836


### Pivoting Data to split the features 


In [139]:
Train_Vehicle = Train_Vehicle.pivot(index='CustomerID', 
                      columns='VehicleAttribute', 
                      values='VehicleAttributeDetails').reset_index()

In [140]:

# Then I will check if the IDs matches the other data
Train_Vehicle.shape[0]

28836

#### Not Important but usefull to get observation of the dataframe 

In [141]:

Train_Vehicle["VehicleMake"].value_counts()

VehicleMake
Saab          2415
Suburu        2313
Nissan        2300
Dodge         2263
Chevrolet     2174
Ford          2158
Accura        2099
BMW           2073
Toyota        1981
Volkswagen    1960
Audi          1952
Jeep          1946
Mercedes      1659
Honda         1493
???             50
Name: count, dtype: int64

In [142]:
Train_Vehicle["VehicleModel"].value_counts()

VehicleModel
RAM               1344
Wrangler          1261
A3                1102
MDX               1054
Jetta             1037
Neon               928
Pathfinder         919
Passat             888
Legacy             887
92x                859
Malibu             828
95                 820
A5                 812
F150               797
Forrestor          784
Camry              771
Tahoe              736
93                 724
Maxima             722
Grand Cherokee     718
Escape             706
Ultima             698
E400               695
X5                 691
TL                 684
Silverado          668
Fusion             650
Highlander         633
Civic              604
ML350              599
Impreza            562
CRV                542
Corolla            530
M5                 509
C300               477
X6                 454
3 Series           436
RSX                397
Accord             310
Name: count, dtype: int64

In [143]:
Train_Vehicle["VehicleYOM"].value_counts()

VehicleYOM
2007    1730
2004    1691
2010    1686
2006    1680
2008    1655
2009    1655
2005    1653
2003    1606
2011    1581
2002    1551
2000    1548
1999    1521
2001    1458
2012    1357
1998    1319
2013    1271
1997    1151
2014     889
1996     851
1995     555
2015     428
Name: count, dtype: int64

### Merging the data and make the customer ID as index

In [144]:
# Merge Demographics + Policy
merged_df = pd.merge(Train_Demographics, Train_Policy, on='CustomerID', how='inner')

# Merge result with Claim
merged_df = pd.merge(merged_df, Train_Claim, on='CustomerID', how='inner')

# Merge result with Claim
merged_df = pd.merge(merged_df, Train_Vehicle, on='CustomerID', how='inner')
# Final merge with target data (ReportedFraud)
final_data = pd.merge(merged_df, Traindata_with_Target[['CustomerID', 'ReportedFraud']], on='CustomerID', how='inner')

# Show final shape
print("Final merged data shape:", final_data.shape)


Final merged data shape: (28836, 42)


In [145]:
print(Traindata_with_Target.columns)
print(Train_Policy.columns)
print(Train_Claim.columns)
print(Traindata_with_Target.columns)
print(Train_Vehicle.columns)


Index(['CustomerID', 'ReportedFraud'], dtype='object')
Index(['InsurancePolicyNumber', 'CustomerLoyaltyPeriod', 'DateOfPolicyCoverage', 'InsurancePolicyState', 'Policy_CombinedSingleLimit', 'Policy_Deductible', 'PolicyAnnualPremium', 'UmbrellaLimit', 'InsuredRelationship', 'CustomerID'], dtype='object')
Index(['CustomerID', 'DateOfIncident', 'TypeOfIncident', 'TypeOfCollission', 'SeverityOfIncident', 'AuthoritiesContacted', 'IncidentState', 'IncidentCity', 'IncidentAddress', 'IncidentTime', 'NumberOfVehicles', 'PropertyDamage', 'BodilyInjuries', 'Witnesses', 'PoliceReport', 'AmountOfTotalClaim', 'AmountOfInjuryClaim', 'AmountOfPropertyClaim', 'AmountOfVehicleDamage'], dtype='object')
Index(['CustomerID', 'ReportedFraud'], dtype='object')
Index(['CustomerID', 'VehicleID', 'VehicleMake', 'VehicleModel', 'VehicleYOM'], dtype='object', name='VehicleAttribute')


### Deleting Uessesary Factours (feature selection)


These couloms seems to be unreleated to the output 


In [146]:
columns_to_drop = [
    "CustomerID",
    "VehicleID",
    'InsuredZipCode',
    'InsurancePolicyNumber',
    'InsuredEducationLevel',
    'InsuredOccupation',
    'InsuredHobbies',
    'Country'
]

In [147]:
final_data = final_data.drop(columns=columns_to_drop)

In [148]:
final_data.head(10)

,InsuredAge,InsuredGender,CapitalGains,CapitalLoss,CustomerLoyaltyPeriod,DateOfPolicyCoverage,InsurancePolicyState,Policy_CombinedSingleLimit,Policy_Deductible,PolicyAnnualPremium,UmbrellaLimit,InsuredRelationship,DateOfIncident,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentState,IncidentCity,IncidentAddress,IncidentTime,NumberOfVehicles,PropertyDamage,BodilyInjuries,Witnesses,PoliceReport,AmountOfTotalClaim,AmountOfInjuryClaim,AmountOfPropertyClaim,AmountOfVehicleDamage,VehicleMake,VehicleModel,VehicleYOM,ReportedFraud
0,35,MALE,56700,-48500,49,1998-10-25,State1,100/300,1000,1632.73,0,not-in-family,2015-02-03,Multi-vehicle Collision,Side Collision,Total Loss,Police,State7,City1,Location 1311,17,3,?,1,0,?,65501,13417,6071,46013,Audi,A5,2008,N
1,36,MALE,70600,-48500,114,2000-11-15,State1,100/300,1000,1255.19,0,not-in-family,2015-02-02,Multi-vehicle Collision,Side Collision,Total Loss,Police,State7,City5,Location 1311,10,3,YES,2,1,YES,61382,15560,5919,39903,Audi,A5,2006,N
2,33,MALE,66400,-63700,167,2001-02-12,State3,500/1000,617,1373.38,0,wife,2015-01-15,Single Vehicle Collision,Side Collision,Minor Damage,Other,State8,City6,Location 2081,22,1,YES,2,3,NO,66755,11630,11630,43495,Volkswagen,Jetta,1999,N
3,36,MALE,47900,-73400,190,2005-04-11,State2,500/1000,722,1337.60,0,own-child,2015-01-19,Single Vehicle Collision,Side Collision,Minor Damage,Other,State9,City6,Location 2081,22,1,YES,2,3,NO,66243,12003,12003,42237,Volkswagen,Jetta,2003,N
4,29,FEMALE,0,-41500,115,1996-10-25,State2,100/300,500,1353.73,4279863,unmarried,2015-01-09,Single Vehicle Collision,Rear Collision,Minor Damage,Fire,State8,City6,Location 1695,10,1,NO,2,1,YES,53544,8829,7234,37481,Toyota,CRV,2010,N
5,28,FEMALE,0,-41500,101,1999-10-24,State2,100/300,500,1334.49,3921366,unmarried,2015-02-07,Single Vehicle Collision,Rear Collision,Minor Damage,Fire,State7,City6,Location 1695,7,1,NO,1,2,?,53167,7818,8132,37217,Toyota,CRV,2011,N
6,57,MALE,67400,0,471,1995-02-18,State3,100/300,512,1214.78,165819,own-child,2015-01-30,Single Vehicle Collision,Front Collision,Minor Damage,Ambulance,State5,City4,Location 1440,20,1,?,0,2,NO,77453,6476,12822,58155,Mercedes,C300,2000,N
7,49,MALE,67400,0,340,1993-02-22,State3,100/300,877,1159.81,5282219,own-child,2015-01-12,Single Vehicle Collision,Front Collision,Minor Damage,Police,State5,City3,Location 1440,18,1,?,0,2,NO,60569,5738,7333,47498,Suburu,C300,2010,N
8,27,FEMALE,56400,-32800,81,1998-05-09,State2,500/1000,2000,989.53,0,own-child,2015-02-06,Multi-vehicle Collision,Front Collision,Minor Damage,Ambulance,State9,City2,Location 1521,3,3,YES,0,0,?,67876,6788,7504,53584,Volkswagen,Passat,1995,N
9,48,MALE,53300,0,328,2014-10-17,State3,250/500,1000,1406.91,0,husband,2015-01-25,Single Vehicle Collision,Side Collision,Major Damage,Police,State7,City2,Location 1596,5,1,YES,1,2,YES,71610,6510,13020,52080,Saab,92x,2004,Y


### Categorical and numerical encoding

#### Renaming to get a numirical data

In [149]:
#These coulomns can be changed easily to numirecal 
final_data['IncidentState'] = final_data['IncidentState'].str.extract(r'(\d+)').astype(int)
final_data['IncidentCity'] = final_data['IncidentCity'].str.extract(r'(\d+)').astype(int)
final_data['IncidentAddress'] = final_data['IncidentAddress'].str.extract(r'(\d+)').astype(int)
final_data["InsurancePolicyState"]= final_data['InsurancePolicyState'].str.extract(r'(\d+)').astype(int)

In [150]:
final_data['ReportedFraud'] = final_data['ReportedFraud'].map({'Y': 1, 'N': 0})

In [151]:
final_data['PropertyDamage'].value_counts()

PropertyDamage
?      10459
NO      9687
YES     8690
Name: count, dtype: int64

In [152]:
#here we cant take the average for the missing data because its a catagorical data and will affect owr results , so we will treat it as a new catagory.
final_data['PoliceReport'] = final_data['PoliceReport'].map({
    'NO': 0,
    'YES': 1,
    '?': 2  # 
})

final_data['PropertyDamage'] = final_data['PropertyDamage'].map({
    'NO': 0,
    'YES': 1,
    '?': 2  # 
})

#### Categorical with get_dummies

In [153]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

OHT_encoder = OneHotEncoder(sparse_output=False)
ORD_encoder = OrdinalEncoder()

In [154]:
#InsuredGender
final_data = pd.get_dummies(final_data, columns=['InsuredGender'], prefix='Gender', drop_first=True)


In [155]:
final_data = pd.get_dummies(final_data, columns=['VehicleMake'], prefix='Make', drop_first=True)


In [156]:
final_data = pd.get_dummies(final_data, columns=['VehicleModel'], prefix='Model', drop_first=True)

In [157]:
final_data = pd.get_dummies(final_data, columns=['InsuredRelationship'], prefix='Relation', drop_first=True)


In [158]:

# Replace "?" and NaN with "Unknown" in TypeOfCollission
final_data['TypeOfCollission'] = final_data['TypeOfCollission'].replace('?', 'Unknown').fillna('Unknown')

# Also fill missing values in the other 3 columns
final_data[['TypeOfIncident', 'SeverityOfIncident', 'AuthoritiesContacted']] = \
    final_data[['TypeOfIncident', 'SeverityOfIncident', 'AuthoritiesContacted']].fillna('Unknown')

# One-hot encode all 4 columns
final_data = pd.get_dummies(final_data, 
                            columns=['TypeOfIncident', 'TypeOfCollission', 'SeverityOfIncident', 'AuthoritiesContacted'], 
                            prefix=['Incident', 'Collision', 'Severity', 'Contacted'], 
                            drop_first=True)

In [159]:
final_data['DateOfPolicyCoverage'] = pd.to_datetime(final_data['DateOfPolicyCoverage'])
final_data['PolicyYear'] = final_data['DateOfPolicyCoverage'].dt.year
final_data['PolicyMonth'] = final_data['DateOfPolicyCoverage'].dt.month
final_data['PolicyDay'] = final_data['DateOfPolicyCoverage'].dt.day
final_data.drop(columns=['DateOfPolicyCoverage'], inplace=True)

In [160]:
# Step 1: Convert to datetime
final_data['DateOfIncident'] = pd.to_datetime(final_data['DateOfIncident'], errors='coerce')

# Step 2: Extract useful date features
final_data['IncidentYear'] = final_data['DateOfIncident'].dt.year
final_data['IncidentMonth'] = final_data['DateOfIncident'].dt.month
final_data['IncidentDay'] = final_data['DateOfIncident'].dt.day
final_data['IncidentDayOfWeek'] = final_data['DateOfIncident'].dt.dayofweek  # Monday=0, Sunday=6

# Optional: Add weekend indicator
final_data['IncidentIsWeekend'] = final_data['IncidentDayOfWeek'].isin([5, 6]).astype(int)

# Step 3: Drop the original column (optional)
final_data.drop(columns=['DateOfIncident'], inplace=True)


## Imputing and Rescaling
In the example below, we demonstrate the effect of utilising two commonly used transformers from sklearn, namely SimpleImputer() and MaxMinScaler().

In [24]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

imputer = SimpleImputer()
scaler = MinMaxScaler()

Make sure All data are numirical

In [170]:
final_data = final_data.astype({col: 'int' for col in final_data.columns if final_data[col].dtype == 'bool'})

In [173]:
final_data['Policy_CombinedSingleLimit'] = final_data['Policy_CombinedSingleLimit'].apply(
    lambda x: eval(x) if isinstance(x, str) and '/' in x else x
)

In [177]:
non_int_cols = [col for col in final_data.columns if not pd.api.types.is_integer_dtype(final_data[col])]
print(non_int_cols)

['Policy_CombinedSingleLimit', 'PolicyAnnualPremium', 'Witnesses', 'AmountOfTotalClaim', 'VehicleYOM']


In [178]:
for col in non_int_cols:
    print(f"{col}: {final_data[col].dtype}")

Policy_CombinedSingleLimit: float64
PolicyAnnualPremium: float64
Witnesses: object
AmountOfTotalClaim: object
VehicleYOM: object


In [180]:
final_data["Witnesses"]

0        0
1        1
2        3
3        3
4        1
        ..
28831    3
28832    2
28833    0
28834    0
28835    3
Name: Witnesses, Length: 28836, dtype: object

## 4. Technique 1
**Create a new subheading for each stage that you do from the following items. Enter your code in the cells below the subheading.**

- Motivation for choosing the technique and schematic figure of the analysis process (100-200 words)
- Setting hyperparameters (rationale)
- Optimising the hyperparameters appropriately
- Performance metrics for training set
- Other items necessary for the technique
- Deal with class imbalance, if necessary, if not done above

### 4.1 Motivation for choosing the technique and schematic figure of the analysis process

### Setting hyperparameters (rationale)

## 5. Technique 2
**Create a new subheading for each stage that you do from the following items. Enter your code in the cells below the subheading.**

- Motivation for choosing the technique and schematic figure of the analysis process (100-200 words)
- Setting hyperparameters (rationale)
- Optimising the hyperparameters appropriately
- Performance metrics for training set
- Other items necessary for the technique
- Deal with class imbalance, if necessary, if not done above

### 5.1 Motivation for choosing the technique and schematic figure of the analysis process

### Setting hyperparameters (rationale)

## 6. Comparison of metrics performance for testing

**Create a new subheading for each stage that you do from the following items. Enter your code in the cells below the subheading.**

- Use of nested cross-validation for both techniques to deal with overfitting model selection and model comparison
- Use appropriate metrics for the testing set
- Use an appropriate model selection visualisation curve (ROC, PR, etc.) that is suitable for the problem at hand
- Checking for overfitting

## 7. Final recommendation of the best model

**Create a new subheading for each item you discuss from the following items.**

- Technical perspective- overfitting discussion, complexity and efficiency
- Business perspective- results interpretation, relevance and balance with a technical perspective

## 8. Conclusion

**Create a new subheading for each item you discuss from the following items.**

- What has been successfully accomplished and what has not
- Reflect back on the analysis and see what you could have done differently if you were to do the project again
- Add a wish list of future work that you would do to take the project forward
